set j "jobs" /1*3/;
alias (j,jp);

parameter p(j) "process time of j"
/ 1  15,    2  6,       3  9/;

scalar M "bigM";
M = sum(j, p(j));
free variable MeanComp   "Mean completion time";
positive variables x(j)  "Start time of job j";
binary variables y(j,jp) "Disjunctive variable for j and jp";

EQUATIONS
MeanCeqn    "Mean completion time",
Disj1(j,jp) "Disjunctive part 1",
Disj2(j,jp) "Disjunctive part 2";

MeanCeqn..   MeanComp =E= sum(j, x(j)+p(j) ) / card(j);
Disj1(j,jp)$(ord(j) lt ord(jp)).. x(j) + p(j)  =L= x(jp) + M*(1-y(j,jp));
Disj2(j,jp)$(ord(j) lt ord(jp)).. x(jp)+ p(jp) =L= x(j)  + M*y(j,jp);

MODEL simplesched /all/;
SOLVE simplesched using MIP minimizing MeanComp;

https://learnche.org/4G3/Mixed-Integer_linear_programming
https://docs.google.com/document/d/1lmBL1Bh251VYb7NmBD2ER-JPD2DSExGR87nu7AWs7oo/edit
https://docs.google.com/document/d/1VZRQ_aaQjMiK_brME9y1dhHYtkhTI_2NLt4PQ51OyhM/edit

In [91]:
from pyomo.environ import *
model = ConcreteModel()

In [92]:
model.j = RangeSet( 3 )
model.jp= RangeSet( 3 )
p={1:15,2:6,3:9}
M=sum(p.values())+10
model.p=Param(model.j,initialize=p)
model.x=Var(model.j,within=NonNegativeReals)
model.y=Var(model.j,model.jp,within=Binary)
model.x.pprint()
model.p.pprint()

x : Size=3, Index=j
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  None :  None : False :  True : NonNegativeReals
      2 :     0 :  None :  None : False :  True : NonNegativeReals
      3 :     0 :  None :  None : False :  True : NonNegativeReals
p : Size=3, Index=j, Domain=Any, Default=None, Mutable=False
    Key : Value
      1 :    15
      2 :     6
      3 :     9


In [93]:
model.Obj = Objective(expr=(sum(model.x[j]+model.p[j] for j in model.j)/max(model.j)) , sense=minimize)

In [94]:
def disj1 (model,j,jp):
    if j<jp:
        return model.x[j] + model.p[j]  <= model.x[jp] + M*(1-model.y[j,jp])
    else:
        return model.x[j] >=0
def disj2 (model,j,jp):
    if j<jp:
        return model.x[jp] + model.p[jp]  <= model.x[j] + M*model.y[j,jp]
    else:
        return model.x[j] >=0
        
model.c1= Constraint( model.j,model.jp, rule=disj1 )
model.c2= Constraint( model.j,model.jp, rule=disj2 )
results = SolverFactory('glpk').solve(model).write()
model.pprint()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17.0
  Upper bound: 17.0
  Number of objectives: 1
  Number of constraints: 19
  Number of variables: 7
  Number of nonzeros: 31
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 9
      Number of created subproblems: 9
  Error rc: 0
  Time: 0.027807950973510742
# ----------------------------------------------------------
#   Solution Information
# ------------------------------------

In [95]:
### Adding a release date
r={1:1,2:1,3:15}
model.r=Param(model.j,initialize=r)
def release(model,j):
    return model.x[j]>=model.r[j]
model.c3=Constraint(model.j,rule=release)
results = SolverFactory('glpk').solve(model).write()
model.pprint()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 20.0
  Upper bound: 20.0
  Number of objectives: 1
  Number of constraints: 22
  Number of variables: 7
  Number of nonzeros: 34
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 7
      Number of created subproblems: 7
  Error rc: 0
  Time: 0.027638912200927734
# ----------------------------------------------------------
#   Solution Information
# ------------------------------------

In [96]:
### Adding a due date for jobs
d={1:20,2:50,3:50}
model.d=Param(model.j,initialize=d)
def due(model,j):
    return model.x[j]+model.p[j] <=model.d[j]
model.c4=Constraint(model.j,rule=due)
results = SolverFactory('glpk').solve(model).write()
model.pprint()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 23.0
  Upper bound: 23.0
  Number of objectives: 1
  Number of constraints: 25
  Number of variables: 7
  Number of nonzeros: 37
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 3
      Number of created subproblems: 3
  Error rc: 0
  Time: 0.03779435157775879
# ----------------------------------------------------------
#   Solution Information
# -------------------------------------